In [1]:
import os
import nengo
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model, Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, \
    Conv3D, MaxPooling2D, Conv1D, AveragePooling1D, LSTM
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold
from keras import backend as K
from sklearn.utils import shuffle

KeyboardInterrupt: 

In [ ]:
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

def get_dataset(dataset_path):
    dataset = np.load(dataset_path)
    features, labels = dataset['features'], dataset['labels']

    features = features.reshape((features.shape[0], 14, -1))
    labels = labels.reshape((-1, 1))
    labels = OneHotEncoder().fit_transform(labels).toarray()

    return features, labels


In [ ]:
def lstm_model():
    model = Sequential([
        LSTM(124, input_shape=(14, 360), activation='relu', return_sequences=True),
        Dropout(0.4),
        BatchNormalization(),
        LSTM(124, activation='relu'),
        Dropout(0.3),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(2, activation='softmax', name='output_layer')
    ])

    return model

def run_network(model: keras.Model, train, val, test, iteration, epochs=30, early_stopping=True):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    # get training, validation and testing data from parameters
    x_train, y_train = train[0], train[1]
    x_val, y_val = val[0], val[1]
    x_test, y_test = test[0], test[1]

    model.fit(
        x=x_train,y=y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1)] if early_stopping else None
    ) # train the model

    eval = model.evaluate(x_test, y_test) # get results
    print('{}. Accuracy: '.format(iteration), eval[1] * 100)
    return eval[1]

In [ ]:
male_dataset = os.path.join('dataset_result', 'men_only.npz')
female_dataset = os.path.join('dataset_result', 'women_only.npz')

def run_dataset(dataset_path):
    features, labels = get_dataset(dataset_path) # get features and labels

    # split the dataset into training and testing data
    x_train, x_test, y_train, y_test = train_test_split(features, labels,
                                                        test_size=0.25, random_state=seed, shuffle=True)

    # function to run cross validation
    def run_cv(early_stopping):
        num_splits = 10 # 10-fold cross validation
        iteration = 1

        results = []

        for train_idx, val_idx in KFold(n_splits=num_splits).split(x_train):
            x_train_curr, y_train_curr = x_train[train_idx], y_train[train_idx]
            x_val_curr, y_val_curr = x_train[val_idx], y_train[val_idx]

            model = lstm_model() # create a fresh lstm model

            result = run_network(model, (x_train_curr, y_train_curr), (x_val_curr, y_val_curr),
                             (x_test, y_test), epochs=30, iteration=iteration, early_stopping=early_stopping)

            results.append(result)
            K.clear_session() # clear tensorflow data
            del model # delete the object
            iteration += 1

        return results # return results

    results_early_stopping = run_cv(True) # get results with early stopping
    results_no_early_stopping = run_cv(False) # get results without early stopping

    return results_early_stopping, results_no_early_stopping


In [ ]:
# Run male dataset
res_es_male, res_no_es_male = run_dataset(male_dataset) # results with and without early stopping for males

In [ ]:
# Run female dataset
res_es_female, res_no_es_female = run_dataset(female_dataset) # results with and without early stopping for females

In [ ]:
# print average and maximum
print('Results:')

print('Female max accuracy early stopping:', max(res_es_female))
print('Female average accuracy early stopping', np.average(res_es_female))
print('Female max accuracy no early stopping:', max(res_no_es_female))
print('Female average accuracy no early stopping:', max(res_no_es_female))

print('Male max accuracy early stopping:', max(res_es_male))
print('Male average accuracy early stopping:', np.average(res_es_male))
print('Male max accuracy no early stopping:', max(res_no_es_male))
print('Male average accuracy no early stopping:', np.average(res_no_es_male))